# NAPARI visualization of UNet Training Data

You can use this notebook to view, modified and save out training data for UNet models

Labels:
+ 0 - background 
+ 1 - GFP/Phase 
+ 2 - RFP


Extra key bindings:
+ 'w' - calculate weightmap
+ '/' - save currently displayed label
+ 's' - save all labels
+ 'o' - output all weightmaps and metadata for tfrecord creation
+ '\>' - grow the label under the mouse cursor
+ '\<' - shrink the label under the mouse cursor

---

```
Authors:
- Alan R. Lowe (a.lowe@ucl.ac.uk)
```

---

## Set up the data path and channel used

In [1]:
# DATA_PATH = '/Users/arl/Dropbox/Data/TrainingData/UNet_training_Scribble'
DATA_PATH = '/home/nathan/analysis/fucci/working_dir/t0_t100'
WEIGHT_AMPLITUDE = 0.
ACQUISION_CHANNELS = ['PHASE']
# ACQUISION_CHANNELS = ['GFP', 'RFP']

---

In [2]:
import os
import re
import enum
import json
import napari
from skimage import io
import numpy as np
from scipy import ndimage

from scipy.ndimage.morphology import distance_transform_edt
from scipy.ndimage import gaussian_filter

import matplotlib.pyplot as plt

In [3]:
@enum.unique
class Channels(enum.Enum):
    BRIGHTFIELD = 0 
    GFP = 1
    RFP = 2
    IRFP = 3
    PHASE = 4
    WEIGHTS = 98
    MASK = 99

In [4]:
def strip_modified_filename(filename):
    if filename.endswith('.modified.tif'):
        stripped_fn = filename[:-len('.modified.tif')]
        return stripped_fn
    return filename

def make_folder(foldername):
    if os.path.exists(foldername):
        return
    os.mkdir(foldername)
    
def file_root(filename):
    FILENAME_PATTERN = r'([a-zA-Z0-9]+)_([a-zA-Z0-9]+)_*.tif'
    grps = re.search(FILENAME_PATTERN, filename)
    return grps

def load_training_data(pth, channels=[Channels.GFP, Channels.RFP]):
    """ load training data for visualisation with napari """
    
   
    
    # find the sets and sort them
    sets = [f for f in os.listdir(pth) if os.path.isdir(os.path.join(pth, f))]
    sets.sort(key = lambda s: int(s[3:]))
    
    def set_filename_format(filename):
        grps = file_root(filename)
        if grps.group(1) in [c.name.lower() for c in all_channels]:
            FILENAME_FORMAT = 2
        else:
            FILENAME_FORMAT = 1
            
        def filename_formatter(filename, channel):
            # assert(channel in [c.name.lower() for c in all_channels])
            grps = file_root(filename)

            return f'{grps.group(FILENAME_FORMAT)}_{channel}.tif'
            # return '{}_{}.tif'.format(*[channel, grps.group(FILENAME_FORMAT)])
        
        return filename_formatter
    
    all_channels = [Channels.MASK, Channels.WEIGHTS]+channels
    files = {k:{'files':[], 'data':[], 'sets':[], 'path':[]} for k in all_channels}
    all_channels.remove(Channels.WEIGHTS)
    
    for s in sets:

        # root_folders
        l_root = os.path.join(pth, s, 'labels')
        
        # check that this folder exists 
        if not os.path.exists(l_root):
            raise IOError(f'{l_root} does not exist. Do you need to rename label -> labels?')

        # get the training label files
        label_files = [f for f in os.listdir(l_root) if f.endswith('.tif')]
        
        # sort to remove unmodified files and replace with the modified files
        unmodified_files, modified_files = [], []
        for i, f in enumerate(label_files):
            if f.endswith('.modified.tif'):
                modified_files.append(strip_modified_filename(f))
            else:
                unmodified_files.append(f)
                
        unmodified_files = list(set(unmodified_files).difference(set(modified_files)))
        label_files = unmodified_files + [f+'.modified.tif' for f in modified_files]
        label_files.sort(key = lambda f: int(f[0:4]))
            #line 68 sorts files numerically 
            
        #print(label_files)
        fnfmt = set_filename_format(label_files[0])
                
        files[Channels.MASK]['path'] += [s+'/labels/'+f for f in label_files]
        files[Channels.MASK]['files'] += [strip_modified_filename(f) for f in label_files]
        files[Channels.MASK]['data'] += [io.imread(os.path.join(l_root, f)) for f in label_files]
        files[Channels.MASK]['sets'] += [s] * len(label_files)
        
        for channel in channels:
            cfiles = [fnfmt(l, channel.name.lower()) for l in label_files]
            files[channel]['path'] += [s+'/'+channel.name.lower()+'/'+f for f in cfiles]
            files[channel]['files'] += cfiles
            files[channel]['data'] += [io.imread(os.path.join(pth, s, channel.name.lower(), f)) for f in cfiles]
            files[channel]['sets'] += [s] * len(label_files)
            
        # now look for weights 
        w_root = os.path.join(pth, s, 'weights') ### change this back to 'weights' and have func to rename previous weight folder?
#         if os.path.exists(w_root):
        wfiles = [fnfmt(l, 'weights') for l in label_files]
        for weight_file in wfiles:
            files[Channels.WEIGHTS]['path'] += [f'{s}/weights/{weight_file}']
            files[Channels.WEIGHTS]['files'] += [weight_file]
            files[Channels.WEIGHTS]['sets'] += [s]
            if os.path.exists(os.path.join(w_root, weight_file)):
                files[Channels.WEIGHTS]['data'] += [io.imread(os.path.join(w_root, weight_file)).astype(np.float32)]
            else:
                print(f'Adding empty weight file: {weight_file}')
                mask_shape = files[channels[0]]['data'][0].shape
                files[Channels.WEIGHTS]['data'] += [np.zeros(mask_shape, dtype=np.float32)]
                                                         
    # now make image stacks 
    for channel in files.keys():
        
        for i, im in enumerate(files[channel]['data']):
            print(channel, files[channel]['path'][i], im.shape, im.dtype)
        
        files[channel]['data'] = np.stack(files[channel]['data'], axis=0)

    return files

In [5]:
channels = [Channels[c.upper()] for c in ACQUISION_CHANNELS]
data = load_training_data(DATA_PATH, channels)

/home/nathan/analysis/notebooks/tifffile.py:2170: UserWarning: failed to import _tifffile.decodepackbits
  warnings.warn("failed to import %s" % module_function)
/home/nathan/analysis/notebooks/tifffile.py:2170: UserWarning: failed to import _tifffile.decodelzw
  warnings.warn("failed to import %s" % module_function)
/home/nathan/analysis/notebooks/tifffile.py:2170: UserWarning: failed to import _tifffile.unpackints
  warnings.warn("failed to import %s" % module_function)


Adding empty weight file: 0000_weights.tif
Adding empty weight file: 0001_weights.tif
Adding empty weight file: 0002_weights.tif
Adding empty weight file: 0003_weights.tif
Adding empty weight file: 0004_weights.tif
Adding empty weight file: 0005_weights.tif
Adding empty weight file: 0006_weights.tif
Adding empty weight file: 0007_weights.tif
Adding empty weight file: 0008_weights.tif
Adding empty weight file: 0009_weights.tif
Adding empty weight file: 0010_weights.tif
Adding empty weight file: 0011_weights.tif
Adding empty weight file: 0012_weights.tif
Adding empty weight file: 0013_weights.tif
Adding empty weight file: 0014_weights.tif
Adding empty weight file: 0015_weights.tif
Adding empty weight file: 0016_weights.tif
Adding empty weight file: 0017_weights.tif
Adding empty weight file: 0018_weights.tif
Adding empty weight file: 0019_weights.tif
Adding empty weight file: 0020_weights.tif
Adding empty weight file: 0021_weights.tif
Adding empty weight file: 0022_weights.tif
Adding empt

Channels.PHASE set1/phase/0000_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0001_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0002_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0003_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0004_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0005_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0006_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0007_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0008_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0009_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0010_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0011_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0012_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0013_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0014_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phase/0015_phase.tif (1024, 1024) uint16
Channels.PHASE set1/phas

In [6]:
def normalize_images(stack):
    normed = stack.astype(np.float32)
    
    for i in range(stack.shape[0]):
        # normed[i,...] = (normed[i,...]-np.mean(normed[i,...])) / np.std(normed[i,...])
        c = normed[i,...]
        p_lo = np.percentile(c,5)
        p_hi = np.percentile(c,99)
        normed[i,...] = np.clip((c - p_lo) / p_hi, 0., 1.)
    return normed

In [7]:
def bounding_boxes(seg):
    lbl, nlbl = ndimage.label(seg)
    class_label, _, minxy, maxxy = ndimage.extrema(seg, lbl, index=np.arange(1, nlbl+1))
    return class_label, minxy, maxxy

In [8]:
seg = np.zeros(data[channels[0]]['data'].shape, dtype=np.uint8)
mask = data[Channels.MASK]['data']
if mask.ndim == 3:
    seg = mask > 0
elif mask.ndim == 4:
    seg[mask[:,0,...]>0] = 1
    seg[mask[:,1,...]>0] = 2

In [9]:
def convert_to_mask(labels, unique_labels=range(1,len(channels)+1)):
    print(unique_labels)
    seg = np.zeros((len(unique_labels),)+labels.shape, dtype=np.uint8)
    for i,l in enumerate(unique_labels):
        seg[i,...] = labels==l
    return np.squeeze(seg)

In [10]:
def save_labels(viewer):
    # get the current image 
    current_slice = viewer.layers[viewer.active_layer].coordinates[0]
    source_set = data[Channels.MASK]['sets'][current_slice]
    source_file = data[Channels.MASK]['files'][current_slice]
    source_fn = os.path.join(source_set, 'labels', source_file)

    # get the current layer
    current_labels = viewer.layers['labels'].data[current_slice,...]
    current_mask = convert_to_mask(current_labels)

    # write out the modified segmentation mask
    new_file = os.path.join(DATA_PATH, source_fn+'.modified.tif')
    print(new_file)
    io.imsave(new_file, current_mask.astype('uint8'))

    print(current_slice, current_labels.shape, new_file)

In [11]:
def save_all_labels(viewer):
        # get the current image 
    current_slice = viewer.layers[viewer.active_layer].coordinates[0]
    #current_slice = viewer.layers[viewer.active_layer].coordinates[1] # prints the x coord
    
    source_set = data[Channels.MASK]['sets'][current_slice]
    source_file = data[Channels.MASK]['files'][current_slice]
    source_fn = os.path.join(source_set, 'labels', source_file)

    # get the current layer
    current_labels = viewer.layers['labels'].data[current_slice,...]
    current_mask = convert_to_mask(current_labels)
    
    # for over all stack
    #for i in range(len(data[Channels.MASK]['files'])):
    for i in range(viewer.layers['labels'].data.shape[0]):
        current_slice = i
        
        source_set = data[Channels.MASK]['sets'][current_slice]
        source_file = data[Channels.MASK]['files'][current_slice]
        source_fn = os.path.join(source_set, 'labels', source_file)
        
        current_labels = viewer.layers['labels'].data[current_slice,...]
        current_mask = convert_to_mask(current_labels)
        
        new_file = os.path.join(DATA_PATH, source_fn+'.modified.tif')
        print(new_file)
        io.imsave(new_file, current_mask.astype('uint8'))

        print(current_slice, current_labels.shape, new_file)


In [12]:
# weightmaps = np.zeros((seg.shape), dtype=np.float32)

def calculate_weightmaps(viewer, w0=WEIGHT_AMPLITUDE, current_slice=0):
    # get the current layer and make it binary
    mask = viewer.layers['labels'].data[current_slice,...].astype(np.bool)
    
    # label the image 
    labelled, n_labels = ndimage.label(mask)
    weight_mask = np.zeros(mask.shape, dtype=np.float32)
    for i in range(1,n_labels+1):
        cell = labelled == i
        not_cell = np.logical_xor(cell, mask) #np.logical_and(labelled != i, labelled > 0)
        mask_diff = gaussian_filter(cell.astype(np.float32), sigma=5) * gaussian_filter(not_cell.astype(np.float32), sigma=5)
        weight_mask += mask_diff

    wmap = w0*weight_mask #* wmap
    
    # normalize it
    wmap += 1.   
    wmap[mask] = 1.
        
    viewer.layers['weightmaps'].data[current_slice,...] = wmap.astype(np.float32)
    viewer.layers['weightmaps'].contrast_limits = (np.min(wmap), np.max(wmap))
    viewer.layers['weightmaps'].visible = True
    
    return wmap

In [13]:
def grow_shrink_label(viewer, grow=True, n_iter=1):
    # get the current image 
    current_slice = viewer.layers[viewer.active_layer].coordinates[0]
    current_labels = viewer.layers['labels'].data[current_slice,...]
    
    cursor_coords = [int(p) for p in viewer.layers[viewer.active_layer].position]
    labelled, _ = ndimage.label(current_labels.astype(np.bool))
    real_label = current_labels[cursor_coords[0], cursor_coords[1]]
    
    if real_label < 1: return
    
    mask = labelled == labelled[cursor_coords[0], cursor_coords[1]]
    if grow:
        mask = ndimage.morphology.binary_dilation(mask, iterations=n_iter)
    else:
        current_labels[mask] = 0
        mask = ndimage.morphology.binary_erosion(mask, iterations=n_iter)
    
    current_labels[mask] = real_label
    viewer.layers['labels'].data[current_slice,...] = current_labels
    viewer.layers['labels']._set_view_slice()

    

In [ ]:
# start napari
with napari.gui_qt():
    viewer = napari.Viewer()
    
    if Channels.GFP in data:
        gfp = normalize_images(data[Channels.GFP]['data'])
        viewer.add_image(gfp, name='GFP', colormap='green', contrast_limits=(0.,1.))
        
    if Channels.RFP in data:
        rfp = normalize_images(data[Channels.RFP]['data'])
        viewer.add_image(rfp, name='RFP', colormap='magenta', contrast_limits=(0.,1.))
        viewer.layers['RFP'].blending = 'additive'
        
    if Channels.PHASE in data:
        phase = normalize_images(data[Channels.PHASE]['data'])
        viewer.add_image(phase, name='Phase', colormap='gray')
    
    if Channels.WEIGHTS in data:
        weightmaps = data[Channels.WEIGHTS]['data']
        viewer.add_image(weightmaps, name='weightmaps', colormap='plasma', visible=True)
    
    viewer.add_labels(seg, name='labels')
    viewer.layers['labels'].opacity = 0.4
    viewer.layers['weightmaps'].blending = 'additive'
    

    @viewer.bind_key('/') #double command
    def k_save_labels(viewer):
        save_labels(viewer)  #need to change the (viewer) bit to save all images   
        
    @viewer.bind_key('s')
    def k_save_all_labels(viewer):
        save_all_labels(viewer)
        
    @viewer.bind_key('w')
    def k_calculate_weightmaps(viewer):
        current_slice = viewer.layers[viewer.active_layer].coordinates[0]
        calculate_weightmaps(viewer, current_slice=current_slice)
        
    @viewer.bind_key('<')
    def k_shrink_label(viewer):
        print('shrink label')
        grow_shrink_label(viewer, grow=False)
    
    @viewer.bind_key('>')
    def k_grow_label(viewer):
        print('grow label')
        grow_shrink_label(viewer, grow=True)
        
    @viewer.bind_key('o')
    def k_output(viewer):
        print('Output all with metadata')
                
        for i in range(viewer.layers['weightmaps'].data.shape[0]):
            
            if np.sum(viewer.layers['weightmaps'].data[i,...]) == 0:
                print(f'Weightmap {i} is empty. Calculating...')
                wmap = calculate_weightmaps(viewer, current_slice=i)
                weight_folder = os.path.join(DATA_PATH, data[Channels.WEIGHTS]['sets'][i], 'weights')
                weight_fn = data[Channels.WEIGHTS]['files'][i]
                print(weight_folder, weight_fn)
                make_folder(weight_folder)
                io.imsave(os.path.join(weight_folder, weight_fn), wmap.astype(np.float32))
                #save_all_labels(viewer) #check if this is possible
            
            
        # write out a JSON file with the data
        jfn = os.path.join(DATA_PATH, 'training_metadata.json')
        jdata = {}
        for channel in data.keys():
            jdata[channel.name.lower()] = data[channel]['path']
            
        with open(jfn, 'w') as json_file:
            json.dump(jdata, json_file, indent=2, separators=(',', ': '))
        
        

range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0000_mask.tif.modified.tif
0 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0000_mask.tif.modified.tif


/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0000_mask.tif.modified.tif is a low contrast image
  from ipykernel import kernelapp as app


range(1, 2)
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0000_mask.tif.modified.tif
0 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0000_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0001_mask.tif.modified.tif
1 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0001_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0002_mask.tif.modified.tif
2 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0002_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0003_mask.tif.modified.tif
3 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0003_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0004_mask.tif.modified.tif
4 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0004_mask.

/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0000_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0001_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0002_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0003_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t10

39 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0039_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0040_mask.tif.modified.tif
40 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0040_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0041_mask.tif.modified.tif
41 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0041_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0042_mask.tif.modified.tif
42 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0042_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0043_mask.tif.modified.tif
43 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0043_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0044_mask.tif.mod

...

/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0000_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0001_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0002_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0003_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t10

41 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0041_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0042_mask.tif.modified.tif
42 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0042_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0043_mask.tif.modified.tif
43 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0043_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0044_mask.tif.modified.tif
44 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0044_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0045_mask.tif.modified.tif
45 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0045_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0046_mask.tif.mod

/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0043_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0044_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0045_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0046_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t10

86 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0086_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0087_mask.tif.modified.tif
87 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0087_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0088_mask.tif.modified.tif
88 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0088_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0089_mask.tif.modified.tif
89 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0089_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0090_mask.tif.modified.tif
90 (1024, 1024) /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0090_mask.tif.modified.tif
range(1, 2)
/home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0091_mask.tif.mod

/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0088_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0089_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0090_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t100/set1/labels/0091_mask.tif.modified.tif is a low contrast image
/home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: /home/nathan/analysis/fucci/working_dir/t0_t10

In [ ]:
# # convert segmentation output labels to multichannel stacks

# p = '/Users/arl/Dropbox/Data/TrainingData/set12'
# files = [f for f in os.listdir(os.path.join(p,'labels')) if f.endswith('.tif')]
# for f in files:
#     mask = io.imread(os.path.join(p, 'labels', f))
#     print(mask.shape)
#     gfp = mask==1
#     rfp = mask==2
#     new_mask = np.stack([gfp, rfp], axis=0)
#     io.imsave(os.path.join(p,f), new_mask.astype('uint8'))